In [131]:
import sqlite3 

In [133]:
conn = sqlite3.connect("heartbound_challenge.db") #Creates a new database, we don't have an existing one to import.
cur = conn.cursor() #We will use this function to interact with the database created.

In [135]:
class GameManager: 
    def __init__(self, conn, cur):
        self.db = conn
        self.cur = cur #Game Manager class is now connected to our database and can modify the data(update/delete) by running sql queries.
        

In [137]:
######Create a 'users' table
cur.execute("""
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY,
    name TEXT,
    category TEXT,
    gender TEXT,
    health INTEGER
)
""")

########Creates a 'sup_power' table. In our scenario, players can activate the super powers they gained from enemies by defeating them in battles but
#####there are consequences.

cur.execute("""
CREATE TABLE IF NOT EXISTS sup_power (
    power_id INTEGER PRIMARY KEY,
    power_name TEXT,
    availability INTEGER,
    user_id INTEGER,
    FOREIGN KEY(user_id) REFERENCES users(user_id)
)
""")

##########Creates a 'game_transaction' table
cur.execute("""
CREATE TABLE IF NOT EXISTS game_transaction (
    battle_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    opponent TEXT,
    power_deployed TEXT,
    power_gained TEXT,
    power_instock INTEGER,          
    consequences TEXT,
    FOREIGN KEY(user_id) REFERENCES users(user_id)
)
""")
conn.commit()
    
 #This is my basic database schema that shows three tables with fields and the relationship between the tables 
##using primary and foreign keys. 



In [191]:
######## Enhanced consequence of borrowing:
cur.execute("""
CREATE TABLE IF NOT EXISTS events (
    id INTEGER PRIMARY KEY,
    name TEXT,
    active INTEGER,
    active_rounds INTEGER,          
    active_duration INTEGER,
    in_penalty INTEGER,
    penalty_rounds INTEGER,
    max_penalty_rounds INTEGER
)
""")
conn.commit()

## active_rounds = # of rounds the capability been active
## active_duration = ideal # of rounds the capability should be active.
## penalty_in = penalty started

In [265]:
#########Adding data to the 'users' table:
users = [
    (11, "Arefin", "User", "Male", 100),
    (25, "Charlotte", "User", "Female", 100),
    (32, "Stanley", "Enemy", "Male", 50),
    (56, "Robert", "Enemy", "Male", 100),
    (24, "Francis", "Enemy", "Male", 100),
    (19, "JY", "Enemy", "Female", 50),
    (27, "Amy", "Enemy", "Female", 50),
    (75, "Cheryl", "Enemy", "Female", 50)
]
cur.execute("DELETE FROM users")
conn.commit() #Prevents data being entered twice. 

cur.executemany("INSERT OR IGNORE INTO users (user_id, name, category, gender, health) VALUES (?, ?, ?, ?, ?)", users)  #Inserting multiple rows of data at once. 
conn.commit()

    print("User '{name}' added.")
except sqlite3.IntegrityError: # The Integrity validation technique prevents duplicate data entries or redundancies.
    print("User ID {user_id} already exists.")

IndentationError: unexpected indent (365780215.py, line 18)

In [195]:
#######Test run:
for row in cur.execute("SELECT * FROM users"):
    print(row) #Prints all the records from the table 'users'.

(11, 'Arefin', 'User', 'Male', 100)
(19, 'JY', 'Enemy', 'Female', 50)
(24, 'Francis', 'Enemy', 'Male', 100)
(25, 'Charlotte', 'User', 'Female', 100)
(27, 'Amy', 'Enemy', 'Female', 50)
(32, 'Stanley', 'Enemy', 'Male', 50)
(56, 'Robert', 'Enemy', 'Male', 100)
(75, 'Cheryl', 'Enemy', 'Female', 50)


In [197]:
########### Adding data to the 'sup_power' table:

sup_power = [
    (42, "Super Speed", 1, 32),
    (45, "Super Strength", 1, 56),
    (65, "Super Durability", 1, 24),
    (50, "Super Speed", 1, 19),
    (62, "Super Strength", 1, 27),
    (72, "Super Durability", 1, 75)
]
cur.execute("DELETE FROM sup_power")
conn.commit() #Avoid repetitive tuples. 

cur.executemany("INSERT OR IGNORE INTO sup_power (power_id, power_name, availability, user_id) VALUES (?, ?, ?, ?)", sup_power)
conn.commit()

In [199]:
##########Adding data to the 'game_transaction' table:
game_transaction = [
    (1, 11, "Stanley", "Initially none", "Super Speed", 1, "Using Speed in 3 consecutive rows, leads to a 50% chance for a missed attack"),
    (2, 11, "Robert", "Super Speed", "Super Strength", 1, "Super speed is active. Using Speed in 3 consecutive rows, leads to a 50% chance for a missed attack"),
    (3, 11, "Francis", "Super Strength", "Super Durability", 1, "Super Strength is active. Over use results in -20 points from the health of user")
]

cur.execute("DELETE FROM game_transaction")
conn.commit()

cur.executemany("INSERT OR IGNORE INTO game_transaction (battle_id, user_id, opponent, power_deployed, power_gained, power_instock, consequences) VALUES (?, ?, ?, ?, ?, ?, ?)", game_transaction)
conn.commit()     

###Context: I mentioned in the storyline that our user has to fight battles and win to gain the capabilities of the enemies and strengthen his 
###capabilities for the upcoming battles. 
### In the user's battle against Robert, he deployed his power 'super speed' that he won from the first battle, defeating Stanley. Deploying super speed
###also comes with its consequence impacting game scenario for example, using the power in 3 consecutive rows can lead to a 50% chance of a missed attack.

In [201]:
#######Adding data to the events table
events = [
    (15, "Super Speed", 1, 0, 2, 0, 0, 2),
    (30, "Durability", 0, 0, 2, 0, 0, 2)
]
cur.execute("DELETE FROM events")
conn.commit()

cur.executemany("INSERT OR IGNORE INTO events (id, name, active, active_rounds, active_duration, in_penalty, penalty_rounds, max_penalty_rounds) VALUES (?, ?, ?, ?, ?, ?, ?, ?)", events)
conn.commit() #Two sample events data entered to experiment our logic.

In [4]:
cur.execute("SELECT name, active, active_rounds, active_duration FROM events WHERE id = ?", (15,))
event = cur.fetchone()   #Extracting an event for experiment. 

if event:
    name, active, active_rounds, active_duration = event  #Now each of these elements of the variable are defined

NameError: name 'cur' is not defined

In [1]:
###Logic to determine if the user would undergo a penalty or not: 
if active == 1:   #---> Active = 1 means capability is active and 0 means inactive.
    active_rounds += 1    #When it's active we add 1 to the active_rounds and check if it triggered penalty in that stage compared to the active_duration(ideal).
    if active_rounds >= active_duration:     ####The main logic deciding the game consequence scenario.
        cur.execute("UPDATE events SET active = 0, in_penalty = 1, penalty_rounds = 0 WHERE active_rounds = ? AND id = ?", (1, 15))
        print("Super Speed has expired. A penalty has started.")
    else:
        cur.execute("UPDATE events SET active_rounds = ? WHERE id = ?", (1, 15))
        print("Super Speed is still active. Round {active_rounds}/{active_duration}.")
         

NameError: name 'active' is not defined

In [ ]:
####Query no. 1: 
###### What power did the user with 'user_id = 11' deploy in his battle with opponent 'Robert'?
cur.execute("CREATE INDEX IF NOT EXISTS idx_game_transaction_user_id_opponent ON game_transaction(user_id, opponent)")
conn.commit() #Indexing to make queries that are sorted by user_id and opponent efficient.
 

cur.execute("SELECT * FROM game_transaction WHERE user_id = ? AND opponent = ?", (11, 'Robert')) #Query run using AND statement.

rows = cur.fetchall()
for row in rows:
    print(row)


### The query above demonstrates that, the user with user_id = 11(Arefin) has used Super Speed in his battle with Robert.

In [155]:
########What is the consequence when user deploys 'super strength' in his fight against Francis?
cur.execute("CREATE INDEX IF NOT EXISTS idx_game_transaction_power_deployed_opponent ON game_transaction(power_deployed, opponent)")
conn.commit() # indexing for faster query process.

cur.execute("SELECT consequences FROM game_transaction WHERE power_deployed = ? AND opponent = ?", ("Super Strength", "Francis"))

rows = cur.fetchall()
for row in rows:
    print(row) # The consequence of 'Super Strength' is demonstrated. 

('Super Strength is active. Over use results in -20 points from the health of user',)


In [225]:
########CLI-driven interaction menu:
def cli_menu():
    gm = GameManager(conn, cur)

    while True:
        print(" Welcome to Heartbound Challenge")
        print("1. Show Available Super Powers")
        print("2. Show consequences in game scenario")
        print("3. Exit")
        choice = input("Select an option: ")

        if choice == "1":
            gm.show_available_powers()
        elif choice == "2":
            print("Be aware of the consequences.")
        elif choice == "3":
            print("Exiting game. Goodbye!")
            break
        else:
            print("Invalid input.")

In [227]:
cli_menu()

 Welcome to Heartbound Challenge
1. Show Available Super Powers
2. Show consequences in game scenario
3. Exit


Select an option:  2


Be aware of the consequences.
 Welcome to Heartbound Challenge
1. Show Available Super Powers
2. Show consequences in game scenario
3. Exit


Select an option:  3


Exiting game. Goodbye!
